<a href="https://colab.research.google.com/github/CyberSentinels05/EDITH/blob/main/Chatbot_Edith_by_Fine_Tuning_a_LLM_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading Requirements

In [ ]:
!pip install streamlit -q
!npm install -g localtunnel
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.545s


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers==4.28.0
!pip install peft
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.0.dev0
    Uninstalling transformers-4.43.0.dev0:
      Successfully uninstalled transformers-4.43.0.dev0


Logging into Hugging Face

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Fine -Tuning the Model

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset,Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

device = torch.device("cpu")

data = load_dataset('HazSylvia/Fitness_Unformatted')
model_id = "openai-community/gpt2-large"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id ,quantization_config=bnb_config)




model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_proj"]  # Change target modules to "c_proj"
)
model = get_peft_model(model, peft_config)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )






tokenizer = GPT2Tokenizer.from_pretrained('openai-community/gpt2-large', pad_token='')
tokenizer.pad_token = tokenizer.eos_token

def preprocess_data(examples):
    combined_text = [f"Question: {q} Answer: {a}" for q, a in zip(examples['Human'], examples['Assistant'])]
    return tokenizer(combined_text, truncation=True, padding='max_length', max_length=128)

tokenized_dataset = data.map(preprocess_data, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

model = GPT2LMHeadModel.from_pretrained('gpt2')
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=300,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=False,
    logging_steps=50,
    output_dir="Fintune_res",
    optim="adamw_torch",
)

# Pass the tokenizer object instead of the tokenized dataset
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset['train'],
     args=training_args,
    data_collator=data_collator,
)



trainer.train()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss
50,3.171000
100,2.639600
150,2.249400
200,2.101700
250,1.974000
300,1.858600


TrainOutput(global_step=300, training_loss=2.332362009684245, metrics={'train_runtime': 6208.1774, 'train_samples_per_second': 0.387, 'train_steps_per_second': 0.048, 'total_flos': 156775219200000.0, 'train_loss': 2.332362009684245, 'epoch': 2.59})

Saving the Fine -Tuned Model

In [ ]:
import os
import json
#model = AutoModelForCausalLM.from_pretrained('/content/Fintune_res')
#tokenizer = AutoTokenizer.from_pretrained('/content/Fintune_res')
model.save_pretrained("/content/Fintune_res")
tokenizer.save_pretrained("/content/Fintune_res")
adapter_config_path = os.path.join("/content/Fintune_res","adapter_config.json") # Add a filename for the config
with open(adapter_config_path, 'w') as f:
    config_dict = peft_config.to_dict()
    # Convert sets to lists for JSON serialization
    for key, value in config_dict.items():
        if isinstance(value, set):
            config_dict[key] = list(value)
    json.dump(config_dict, f)


Inference

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = torch.device("cpu")

# Load model and tokenizer
model_name_or_path = "/content/Fintune_res"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

# Get user input
user_input = input("Enter your prompt: ")

# Encode the input and generate a response
inputs = tokenizer.encode(user_input, return_tensors="pt")
attention_mask = torch.ones(inputs.shape, dtype=torch.long)

# Hook to capture position_ids
position_ids_list = []
def capture_position_ids(module, input, output):
    position_ids_list.append(input[0])  # Assuming position_ids is the first element in the input tuple

# Register the hook on the relevant layer
# You might need to inspect the model architecture to find the exact layer
# where position_ids are used. This is often within the attention layers.
# For example, if it's in the first attention layer:
model.transformer.h[0].attn.register_forward_hook(capture_position_ids)

outputs = model.generate(
    inputs,
    attention_mask=attention_mask,
    max_length=89,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    eos_token_id=tokenizer.eos_token_id


)

# Remove the hook after capturing position_ids
model.transformer.h[0].attn._forward_hooks.clear()

# Access the captured position_ids
position_ids = position_ids_list[0]
print("Captured Position IDs shape:", position_ids.shape)
print("Captured Position IDs:", position_ids)

# Decode and print the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Enter your prompt: give me a nutrition tip 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Captured Position IDs shape: torch.Size([1, 6, 768])
Captured Position IDs: tensor([[[ 0.1136, -0.1155,  0.0191,  ..., -0.1135,  0.0689,  0.0336],
         [ 0.1805, -0.0451, -0.0579,  ..., -0.1267,  0.0858,  0.0387],
         [-0.0545, -0.0492,  0.0234,  ...,  0.0568,  0.0821, -0.0644],
         [-0.1057, -0.0627,  0.0301,  ...,  0.2952, -0.0195,  0.1691],
         [ 0.2285, -0.0089,  0.1153,  ..., -0.2044,  0.0712,  0.0129],
         [ 0.1206, -0.0972,  0.1140,  ..., -0.0583, -0.1743,  0.1138]]])
give me a nutrition tip  Answer:  eat a balanced diet with plenty of protein and healthy fats 

 eat proteinrich foods like lean proteins and lean carbs 
 avoid processed and sugary foods   avoid sugared foods such as processed sugar and saturated fats

 avoid refined carbs and sugar  especially if youre a vegetarian or a keto diet  you can still get some of these carbs from foods that are high in saturated fat


 Deployment of EDITH using Streamlit

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name_or_path = "/content/Fintune_res"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

# Streamlit app with HTML and CSS
st.set_page_config(page_title="LLM Deployment", page_icon=":robot_face:", layout="wide")

# Custom CSS for enhanced styling
st.markdown("""
    <style>
    body {
        background-color: #ffffff;
        font-family: 'Arial', sans-serif;
        color: #333;
    }
    .main {
        background-color: #ffffff;
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        max-width: 800px;
        margin: 20px auto;
    }
    .title {
        color: #ffffff;
        text-align: center;
        padding: 20px;
        font-size: 2.5em;
        border-bottom: 2px solid #2980b9;
        margin-bottom: 20px;
        background-color: #2c3e50;
        border-radius: 10px;
    }
    .text-input-wrapper {
        width: 100%;
        padding: 15px;
        font-size: 18px;
        border: 1px solid #000000;
        border-radius: 10px;
        box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.1);
        margin-bottom: 20px;
        background-color: #fff;
    }
    .button {
        background-color: #2980b9;
        color: white;
        padding: 15px 20px;
        text-align: center;
        display: block;
        font-size: 18px;
        margin: 20px 0;
        cursor: pointer;
        border: none;
        border-radius: 10px;
        box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
        transition: background-color 0.3s ease;
        text-align: center;
        text-decoration: none;
    }
    .button:hover {
        background-color: #3498db;
    }
    .response-box {
        margin-top: 20px;
        padding: 20px;
        background-color: #ffffff;
        border: 1px solid #000000;
        border-radius: 10px;
        box-shadow: 0px 2px 5px rgba(0,0,0,0.15);
    }
    .response-text {
        font-size: 1.1em;
        line-height: 1.6;
        color: #34495e;
    }
    </style>
""", unsafe_allow_html=True)

# Title
st.markdown("<h1 class='title'>CHAT WITH EDITH </h1>", unsafe_allow_html=True)

# Text input with styling
st.markdown('<div class="text-input-wrapper">', unsafe_allow_html=True)
user_input = st.text_area(
    "Enter your prompt:",
    "",
    key="input",
    height=100,
    max_chars=None,
    placeholder="Type here...",
    help="Type a prompt to get a response",
)
st.markdown('</div>', unsafe_allow_html=True)

# Generate response when button is clicked
if st.button("ASK", key="button", help="Click to generate a response"):
    if user_input:
        # Encode the input and generate a response
        inputs = tokenizer.encode(user_input, return_tensors="pt")
        attention_mask = torch.ones(inputs.shape, dtype=torch.long)

        # Dynamic max token calculation
        base_length = 50  # Base length for short inputs
        length_multiplier = 3.0  # Adjust this based on desired response length relative to input
        estimated_max_length = int(base_length + len(user_input.split()) * length_multiplier)


        outputs = model.generate(
            inputs,
             attention_mask=attention_mask,
              max_length=100,
             num_return_sequences=1,
            no_repeat_ngram_size=2,
             eos_token_id=tokenizer.eos_token_id


)

        # Decode and print the output
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Ensure the response is complete
        while response[-1] != ' ' and len(response) < 2 * estimated_max_length:
            outputs = model.generate(
                inputs=outputs,
                attention_mask=attention_mask,
                max_length=len(response.split()) + 50,
                num_return_sequences=1,
                no_repeat_ngram_size=2,
                early_stopping=True,
                #pad_token_id=tokenizer.eos_token_id,
                #temperature=0.6,
                #repetition_penalty=2.2
            )
            additional_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            response += additional_response[len(response):]

        # Display the response
        st.markdown('<div class="response-box">', unsafe_allow_html=True)
        st.markdown(f'<div class="response-text">{response}</div>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)


Overwriting app.py


Running Streamlit using Localtunnel

In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.196.179.161


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.179.161:8501

npx: installed 22 in 4.621s
your url is: https://brown-donuts-worry.loca.lt
  Stopping...
^C
